In [ ]:
# coding: utf-8
import pandas as pd
import numpy as np
import time
import math
from tqdm import tqdm
import matplotlib.pyplot as plt

from package.utils import KPIPoint
from package.utils import KPISet
from package.utils import Transformer
from package.HotSpot import HotSpot

#### 加载前2周数据

In [ ]:
timestamp_strat = 1535731200
timestamp_end = 1536940500 #1535731200 + 5 * 60#
timestamp_interval = 5 * 60
file_path = '../2019AIOps_data/'
kSet = Transformer().transformKPIData2KPISet(file_path, timestamp_strat, timestamp_end, timestamp_interval)
kSet.save('../result/metadata/KPISet')

#### 加载后2周数据（注入了异常）

In [ ]:
timestamp_strat = 1536940800
timestamp_end = 1538150100 #1536940800 + 5 * 60#
timestamp_interval = 5 * 60
file_path = '../2019AIOps_data_test1/'
kSet_test = Transformer().transformKPIData2KPISet(file_path, timestamp_strat, timestamp_end, timestamp_interval)
kSet_test.save('../result/metadata/KPISetTest')

#### 使用前2周数据预测后2周数据，此处可以优化

In [ ]:
# 使用前两周数据预测本周数据
I = 300
T = I * 288
train_timestamp_start = 1535731200
train_timestamp_end = 1536940500
for timestamp in tqdm(kSet_test._KPIPoints):
    tw = math.floor((timestamp - 16 * 3600) % (3600 * 24) / I)
    # print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(timestamp)), tw)
    for leaf in kSet_test._KPIPoints[timestamp]._leaf:
        ts_true = kSet.get_ts_leaf(leaf=leaf, t1=train_timestamp_start + tw * I, t2=train_timestamp_end, delta=T)[
            'true']
        predict = np.mean(ts_true)
        kSet_test._KPIPoints[timestamp]._leaf[leaf][1] = predict

#### 保存数据
kSet_test.save('../result/metadata/KPISetTestPredict')

#### 异常定位

In [ ]:
# 加载数据集
# kSet_test = KPISet({}, {})
# kSet_test.load('../result/metadata/KPISetTestPredict')

# 读取异常时间戳
outlier = pd.read_csv('../Anomalytime_data_test1.csv')
outlier = outlier['timestamp'].tolist()

# HotSpot参数
ps_threshold = 0.98  # 潜在得分阈值
ep_threshold = 0.01  # 解释力阈值
max_iter = 10  # MCTS最大迭代次数

res = {}
res['timestamp'] = []
res['set'] = []
sTime = time.time()
for timestamp in tqdm(outlier):
    ts = timestamp / 1000
    kPoint = kSet_pred._KPIPoints[ts]
    layer_max = len(kPoint._attribute_names)
    hotSpot = HotSpot(kPoint, layer_max, ps_threshold, ep_threshold, max_iter)
    rootCauseSet = hotSpot.find_root_cause_set_revised()
    res['timestamp'].append(timestamp)
    sets = []
    for ele in rootCauseSet[0][0]:
        sets.append("&".join(ele))
    res['set'].append(';'.join(sets))
eTime = time.time()
print('runtime %fs' % (eTime - sTime))
res = pd.DataFrame(res)
resPath = '../result/submit_pst%f_ept%f_iter%d.csv' % (ps_threshold, ep_threshold, max_iter)
res.to_csv(resPath, index=False)